In [195]:
import pacepredictor as pp
import importlib
import plotly.express as px
import pandas as pd
importlib.reload(pacepredictor)

<module 'pacepredictor' from '/home/adam/concept2-pace-predictor/Concept2-Pace-Predictor/pacepredictor.py'>

# Load Data

In [196]:
file_path = "/home/adam/concept2-logbook-scraper/analysis/"
file_5_2_10 = "5_2_10.csv"

pp_5_2_10 = pp.Predictor(f"{file_path}{file_5_2_10}", random_state=42, dummies=False)

pp_5_2_10.df

,5000_time,2000_time,10000_time,adaptive rowing category,age,height,weight,weight_class
4,1086.8,416.4,2222.6,able,56,188.0,97.1,H
11,1075.6,388.7,2221.0,able,48,185.4,93.0,H
17,1006.9,387.3,2051.8,able,45,185.4,73.9,L
24,1334.1,523.9,2841.8,able,84,172.7,73.0,L
37,1849.6,754.6,3999.7,able,79,172.7,65.8,L
...,...,...,...,...,...,...,...,...
65125,1164.4,450.6,2711.6,able,44,193.0,99.8,H
65272,1463.0,632.6,3418.2,able,43,165.1,89.8,H
65299,1364.3,505.0,2828.1,able,58,162.6,59.0,L
65330,1215.6,458.9,2531.2,able,45,182.9,92.5,H


# Predictor

Create a regression model to predict 2000m time

## Visualise the Relationship Between the Workouts

In [197]:
fig = px.scatter_3d(pp_5_2_10.df, x="2000_time", y="5000_time", z="10000_time", title="2000m vs 5000m vs 10000m")

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[pp_5_2_10.df["2000_time"].min(),900],),
        yaxis = dict(nticks=4, range=[pp_5_2_10.df["5000_time"].min(),2500],),
        zaxis = dict(nticks=4, range=[pp_5_2_10.df["10000_time"].min(),5000],)))

fig.show()

## Split Data

Need a train, validation and test set.

70% training data. 15% test and 15% validate

In [198]:
pp_5_2_10.split_data()

## Plot Linear Regression

In [201]:
import statsmodels.formula.api as smf

y_col = "2000_time"
formula = pp.create_lr_formula(y_col,pp_5_2_10.df_train.loc[:, pp_5_2_10.df.columns != y_col])

# model = smf.ols(formula=formula, data=self.df_train).fit()





In [202]:
formula

'2000_time ~ + 5000_time + 10000_time + adaptive rowing category + age + height + weight + weight_class'